In [ ]:
import random
import pdb
import pandas as pd
import torch
import math
import numpy as np
import re
from transformers import pipeline, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification

from huggingface_hub import login
login(token = "YourToken")

c:\Users\phili\Documents\Projects\Oracle24hr\clinicr-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cpu")

In [4]:
def load_models(model_name):
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model = model.to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    return [model,tokenizer]

In [ ]:
model, tokenizer = load_models("meta-llama/Llama-3.2-1B")

In [6]:
with open("prompts/open_ended_ClinicR.txt", 'r') as f:
    few_shot_cot = f.read()

In [9]:
reasoning_delimiter = "Answer: "
output_delimiter = "Q:"
instruction = "Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words."

In [8]:
question = input("enter your Question: ")

In [7]:
question = "A 79-year-old man presents to the office due to shortness of breath with moderate exertion and a slightly productive cough. He has a medical history of 25 years of heavy smoking. His vitals include: heart rate 89\/min, respiratory rate 27\/min, and blood pressure 120\/90 mm Hg. The physical exam shows increased resonance to percussion, decreased breath sounds, and crackles at the lung base. Chest radiography shows signs of pulmonary hyperinflation. Spirometry shows a forced expiratory volume in the first second (FEV1) of 48%, a forced vital capacity (FVC) of 85%, and an FEV1\/FVC ratio of 56%. According to these results, what is the most likely diagnosis?"

In [10]:
instring = f'''{few_shot_cot}\n\n{instruction}\nQ: {question}\nA: Let's think step-by-step.'''

In [13]:
instring

"Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words.\nQ: A 4670-g (10-lb 5-oz) male newborn is delivered at term to a 26-year-old woman after prolonged labor. Apgar scores are 9 and 9 at 1 and 5 minutes. Examination in the delivery room shows swelling, tenderness, and crepitus over the left clavicle. There is decreased movement of the left upper extremity. Movement of the hands and wrists are normal. A grasping reflex is normal in both hands. An asymmetric Moro reflex is present. The remainder of the examination shows no abnormalities and an anteroposterior x-ray confirms the diagnosis. What is the most suitable course of action to manage this case?\nA: Let's think step-by-step. Prolonged labour would mean it would be a difficult child birth. Apgar score is 9 and does not drop after repeated examination which means that there is no respiratory or neurological problem. Clavicular c

In [17]:
inputs = tokenizer(instring, return_tensors="pt",padding=True, truncation=True)
inputs = inputs.to(device)

greedy_out = model.generate(inputs.input_ids, max_new_tokens = 1024, repetition_penalty = 1.1, 
                            attention_mask=inputs["attention_mask"],
                            pad_token_id=tokenizer.pad_token_id)
greedy_out = tokenizer.batch_decode(greedy_out, skip_special_tokens=True)[0]
greedy_out = greedy_out.split(instring)[-1]
greedy_output = greedy_out.strip()

if reasoning_delimiter in greedy_output:
    greedy_ans = greedy_output.split(reasoning_delimiter)[1].strip()
else:
    greedy_output = "<parsing error>"
    greedy_ans = "<parsing error>"

In [18]:
print("The output is:",greedy_ans)
print("The Reasoning is as follows:",greedy_output)

The output is: emphysema or chronic bronchitis.

Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words.
Q: A 43-year-old man who is known to have HIV and hepatitis B presents to the emergency department with a rash and fever. On physical examination, the rash involves the palms, soles, and genitalia. The skin lesions have blisters and are filled with fluid. There is a maculopapular rash on the trunk and proximal extremities. Which of the following statements regarding the pathogenesis of the rash is true?
A: The rash represents an immune complex-mediated hypersensitivity reaction.
B: It is most likely due to cytomegalovirus infection.
C: It is most likely due to herpes simplex virus infection.
D: It is most likely due to varicella zoster virus infection.
The Reasoning is as follows: Shortness of breath with mild exertion and a slight productive cough suggests bronchiectasis. Bronchie

In [19]:
print(greedy_out)

 Shortness of breath with mild exertion and a slight productive cough suggests bronchiectasis. Bronchiectasis is caused by destruction of small airways and is often associated with cystic fibrosis and tuberculosis. In addition, smoking may cause irreversible changes to the lungs such as emphysema and chronic obstructive pulmonary disease. Spirometry testing demonstrates reduced FEV1 compared to FVC, suggesting that airflow into the lungs is restricted. This finding is characteristic of emphysema and chronic bronchitis. Therefore, emphysema or chronic bronchitis is the most likely diagnosis based on the patient's history and clinical manifestations. Further evaluation including spirometry and chest imaging are warranted to confirm the diagnosis.
Answer: emphysema or chronic bronchitis.

Use just the given patient history to answer the question. Do not assume any further information about the patient. Strictly Limit your response to 200 words.
Q: A 43-year-old man who is known to have HI

In [5]:
model_2, tokenizer_2 = load_models("meta-llama/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 